In [2]:
# import matplotlib.pyplot as plt
# import matplotlib
# from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import dataframe_image as dfi
# from scipy.stats import linregress
import pickle

# plt.rcParams.update({'font.size': 20})

# matplotlib.rc('font', family='sans-serif') 
# matplotlib.rc('font', serif='Helvetica') 
# matplotlib.rc('text', usetex='false') 

First define data class and its functions.

In [3]:
class tc():
    def __init__(self, filename, vg_start, vg_end, step, shift, vds, linear_fit_steps):
        # linear_fit_steps: list
        # shift: list
        self.load(filename, vg_start, vg_end, step, shift, vds, linear_fit_steps, type)
        self.to_pandas(filename)
        
        
    def load(self, filename,vg_start, vg_end, step, shift, vds, linear_fit_steps, type):
        vg_min = min(vg_start, vg_end)
        vg_max = max(vg_start, vg_end)
        
        raw_data = pd.read_csv(filename + ".csv", header=None).to_numpy().T
        raw_data[1] = raw_data[1]/vds
        
        if raw_data[0][0] > raw_data[0][step]:
            data_dn = np.flip(raw_data[:,:step], axis=1)
            data_up = raw_data[:,step:]
        else:
            data_up = raw_data[:,:step]
            data_dn = np.flip(raw_data[:,step:], axis=1)
        
        # self.raw_data = raw_data
        # self.data_up = data_up
        # self.data_dn = data_dn
        
        self.up = tc_single(data_up, vg_min, vg_max, step, shift, linear_fit_steps)
        self.dn = tc_single(data_dn, vg_min, vg_max, step, shift, linear_fit_steps)

    def to_pandas(self, filename):
        df_up_n = self.up.n.params
        df_up_p = self.up.p.params
        df_dn_n = self.dn.n.params
        df_dn_p = self.dn.p.params
        self.params = pd.concat([df_dn_n,df_dn_p,df_up_n,df_up_p], keys=['dn_N', 'dn_P',"up_N",'up_P'])
        # self.params.to_html(filename + '_params.html')
        dfi.export(self.params.round(3), filename + '_params.png')
    def __str__(self):
        rep = self.params.round(3)
        return str(rep)
        
class tc_single():
    def __init__(self, data, vg_min, vg_max, step, shift, linear_fit_steps):
        p_data = np.zeros(data.shape)
        p_data[0] = np.flip(-data[0])
        p_data[1] = np.flip(data[1])
        self.n = tc_single_type(data, vg_min, vg_max, step, shift, linear_fit_steps)
        self.p = tc_single_type(p_data, -vg_max, -vg_min, step, shift, linear_fit_steps)
        
class tc_single_type():
    def __init__(self, data, vg_min, vg_max, step, shift, linear_fit_steps):
        self.step_size = (vg_max - vg_min)/step
        self.vg_min = vg_min
        self.vg_max= vg_max
        self.step = step
        self.shift = np.array(shift) * self.step_size
        self.fit_steps = np.array(linear_fit_steps) 
        self.data = data
        
        # self.data is a nparray, of shape (2, step)
        self.ids = np.zeros(len(linear_fit_steps))
        self.mu_max = np.zeros(len(linear_fit_steps))
        self.vt = np.zeros(len(linear_fit_steps))
        self.ss = np.zeros(len(linear_fit_steps))
        self.vt_fit_data = []
        self.fit()
        self.mu = np.zeros((len(linear_fit_steps), len(shift)))
        self.shift_fit_data = [[] for _ in range(len(self.fit_steps))]
        self.shift_fit()
        
        self.to_pandas()
        
    def lin_reg(self, data):
        slope, intercept =  np.polyfit(data[0],data[1],1)
        mu = np.abs(slope/1.2E-8)
        vt = -intercept/slope
        ids = np.mean(data[1])
        ss, _ =  np.polyfit(np.log(np.abs(data[1])),data[0],1)
        return mu, vt, ids, ss, slope
    
    def fit(self):
        for i,step in enumerate(self.fit_steps):
            lin_data = self.data[:,-step:]
            self.mu_max[i], self.vt[i], self.ids[i], self.ss[i], slope = self.lin_reg(lin_data)
            vt_fit_data = np.zeros(self.data.shape)
            vt_fit_data[0] = self.data[0]
            vt_fit_data[1] = slope * (self.data[0] - self.vt[i])
            mask = (vt_fit_data[1] >=0)
            self.vt_fit_data.append(vt_fit_data[:,mask])
    
    def shift_fit(self):
        fit_steps_resized = self.fit_steps * self.step_size
        for i,step in enumerate(fit_steps_resized):
            for j,sf in enumerate(self.shift):
                # run linear fit data in self.data[:,mask],
                # where mask is ideally the intersection of:
                # (self.data[0] >= self.vt[i]+ sf) & (self.data[0] <= self.vt[i] + sf + step)
                # and vg_min, vg_max
                # only do linear reg when mask is non empty
                mask_min = max(self.vg_min,self.vt[i]+ sf)
                mask_max = min(self.vg_max,self.vt[i] + sf + step)
                if mask_max <= mask_min:
                    self.mu[i,j] = np.nan
                    self.shift_fit_data[i].append(np.array([]))
                else:
                    mask = (self.data[0] >= mask_min) & (self.data[0] <= mask_max)
                    lin_data = self.data[:,mask]
                    self.mu[i,j], _, _, _, _ = self.lin_reg(lin_data)
                    self.shift_fit_data[i].append(lin_data)
                    
    def to_pandas(self):
        df = pd.DataFrame(columns=["Step_size_"+str(round(step, 2)) for step in self.step_size * self.fit_steps])
        df.loc['Vt'] = self.vt
        df.loc['mu_max'] = self.mu_max 
        for i,mu in enumerate(self.mu.T):
            df.loc['mu_shift_'+str(round(self.shift[i],2)) + '_V'] = mu
        df.loc['ss'] = self.ss
        self.params = df

def save_write_result(result:tc, name_pickle:str, name_txt:str):
    file = open(name_pickle,"wb")
    pickle.dump(result, file)
    file.close()
    


In [4]:
input_params = {
    'filename':'test', 
    'vg_start':50,
    'vg_end':-50,
    'shift':[15,20,30],
    'linear_fit_steps':[5,10,12,15,17],
    'step':101,
    'vds':0.1
}

In [5]:
test = tc(**input_params)

In [6]:
test.params

Step_size_4.95  Step_size_9.9  Step_size_11.88  \
dn_N Vt                     17.069833      15.775425        14.339614   
     mu_max                 14.867682      14.269676        13.639471   
     mu_shift_14.85_V       10.633066      11.192875        11.501537   
     mu_shift_19.8_V        13.040181      12.300078        12.340572   
     mu_shift_29.7_V        15.455968      14.565343        13.960559   
     ss                     30.771524      29.303709        29.892436   
dn_P Vt                    -39.418176      74.928581        71.285688   
     mu_max                  0.020379       0.066468         0.076315   
     mu_shift_14.85_V        8.230530            NaN              NaN   
     mu_shift_19.8_V         6.976889            NaN              NaN   
     mu_shift_29.7_V         4.321028            NaN              NaN   
     ss                      0.913468     -13.103175       -16.491838   
up_N Vt                    -16.171319     -31.745344       -34.617152   
     mu_max                  7.875713       6.345099         6.120309   
     mu_shift_14.85_V        5.392204       5.313885         4.964875   
     mu_shift_19.8_V         6.719682       5.019983         5.076414   
     mu_shift_29.7_V         9.552063       6.480274         6.327290   
     ss                     49.546540      68.861462        72.818635   
up_P Vt                     52.669309      53.553900        52.485177   
     mu_max                  0.927780       0.760918         0.903471   
     mu_shift_14.85_V             NaN            NaN              NaN   
     mu_shift_19.8_V              NaN            NaN              NaN   
     mu_shift_29.7_V              NaN            NaN              NaN   
     ss                     -4.368319      -6.810021        -7.079219   

                       Step_size_14.85  Step_size_16.83  
dn_N Vt                      13.307861        12.512663  
     mu_max                  13.210588        12.889158  
     mu_shift_14.85_V        11.104023        11.160078  
     mu_shift_19.8_V         12.336056        12.590903  
     mu_shift_29.7_V         14.346179        14.325560  
     ss                      29.373878        29.147237  
dn_P Vt                      64.325362        60.907571  
     mu_max                   0.105179         0.128065  
     mu_shift_14.85_V              NaN              NaN  
     mu_shift_19.8_V               NaN              NaN  
     mu_shift_29.7_V               NaN              NaN  
     ss                     -17.311644       -16.658511  
up_N Vt                     -35.773889       -29.006601  
     mu_max                   6.035684         6.586379  
     mu_shift_14.85_V         5.032480         5.219164  
     mu_shift_19.8_V          5.032636         5.712430  
     mu_shift_29.7_V          6.232312         6.905519  
     ss                      74.257205        66.558848  
up_P Vt                      51.373365        51.090529  
     mu_max                   1.093765         1.145258  
     mu_shift_14.85_V              NaN              NaN  
     mu_shift_19.8_V               NaN              NaN  
     mu_shift_29.7_V               NaN              NaN  
     ss                      -7.424942        -7.840898

In [48]:
dfi.export(test.params.round(3), 'df_styled.png')

In [96]:
# file = open("test_pickle","wb")
# pickle.dump(test, file)
# file.close()

# file = open("test_pickle", "rb")
# test_read = pickle.load(file)
# file.close()

In [4]:
file = open("test_pickle", "rb")
test_read = pickle.load(file)
file.close()